# Compound shell

## Import packages

In [6]:
import pandas as pd
import numpy as np
from astropy import constants
from astronavigation.deflection import *
from astronavigation.planets import Body, SolarSystem

## Useful constants

In [7]:
pc = constants.pc.to('km').value
AU = constants.au.to('km').value
c = constants.c.to('km/s').value
eps = 1/c

v_null = np.array([0, 0, 0])

# Create Solar System
ss = SolarSystem()

jupiter = ss.getPlanet('jupiter')
jup_mass = jupiter.mass
jup_radius = jupiter.radius

r_jup_arc = 20  # mean jupiter radius in arcseconds

## Read and clean data

Read data

In [8]:
# path
path = 'Stars_GareqEvent2017_oneTransit_new2.dat'

# read, use delimiter='\s+' to choose every white space as delimiter
stars_data = pd.read_table(path, delimiter='\s+')

Evaluate impact parameter

In [9]:
star = np.array([stars_data['etaS[deg]'], stars_data['zetaS[deg]']])
jup = np.array([stars_data['etaJup[deg]'], stars_data['zetaJup[deg]']])

stars_data['b_J'] = np.linalg.norm(star - jup, axis=0) * 3600 / r_jup_arc
stars_data

tAF5         tAFx               starId  etaS[deg]  sig_eta[mas]  \
0    2017.145239  2017.145239  3631075715518049024   0.447504      0.099969   
1    2017.145239  2017.145239  3631485528413582336  -0.587043      0.072845   
2    2017.145239  2017.145239  3631191099815174272   0.115714      0.091820   
3    2017.145239  2017.145239  3631256894418771584  -0.145632      0.096824   
4    2017.145239  2017.145239  3631266072764288000  -0.265168      0.103025   
..           ...          ...                  ...        ...           ...   
137  2017.145239  2017.145240  3631095407943986048   0.228960      0.071818   
138  2017.145239  2017.145240  3631244559272698112  -0.088713      0.049222   
139  2017.145239  2017.145240  3631244937229820800  -0.079046      0.054070   
140  2017.145239  2017.145240  3631089188831339264   0.347718      0.055859   
141  2017.145239  2017.145240  3631096438736162944   0.142424      0.046130   

     zetaS[deg]  sig_zeta[mas]  etaJup[deg]  zetaJup[deg]  etaSpin[deg]  ...  \
0      0.149847       0.372775     0.012973      0.020895     20.471557  ...   
1      0.052752       0.312333     0.012973      0.020895     20.471557  ...   
2      0.075482       0.339177     0.012973      0.020895     20.471557  ...   
3     -0.074359       0.311625     0.012973      0.020895     20.471557  ...   
4     -0.023983       0.389037     0.012973      0.020895     20.471557  ...   
..          ...            ...          ...           ...           ...  ...   
137    0.058662       0.318960     0.012963      0.020873     19.823930  ...   
138   -0.037527       0.220813     0.012963      0.020873     19.823930  ...   
139    0.050917       0.226070     0.012963      0.020873     19.823930  ...   
140    0.045807       0.233511     0.012963      0.020873     19.823930  ...   
141    0.081247       0.184189     0.012963      0.020873     19.823930  ...   

      gaiaPosX[m]   gaiaPosY[m]   gaiaPosZ[m]  long_jupGC[deg]  \
0   -1.338477e+11  6.053463e+10  2.630360e+10       201.426291   
1   -1.338477e+11  6.053463e+10  2.630360e+10       201.426291   
2   -1.338477e+11  6.053463e+10  2.630360e+10       201.426291   
3   -1.338477e+11  6.053463e+10  2.630360e+10       201.426291   
4   -1.338477e+11  6.053463e+10  2.630360e+10       201.426291   
..            ...           ...           ...              ...   
137 -1.338483e+11  6.053366e+10  2.630318e+10       201.426269   
138 -1.338483e+11  6.053366e+10  2.630318e+10       201.426269   
139 -1.338483e+11  6.053366e+10  2.630318e+10       201.426269   
140 -1.338483e+11  6.053366e+10  2.630318e+10       201.426269   
141 -1.338483e+11  6.053366e+10  2.630318e+10       201.426269   

     lat_jupGC[deg]  long_gaia[deg]  lat_gaia[deg]  starRA[deg]  starDec[deg]  \
0         -7.396121      155.664430      10.151667   201.552776     -7.831722   
1         -7.396121      155.664430      10.151667   201.463499     -6.796371   
2         -7.396121      155.664430      10.151667   201.480483     -7.499324   
3         -7.396121      155.664430      10.151667   201.331641     -7.236695   
4         -7.396121      155.664430      10.151667   201.383427     -7.117590   
..              ...             ...            ...          ...           ...   
137       -7.396111      155.664862      10.151503   201.462561     -7.612432   
138       -7.396111      155.664862      10.151503   201.368255     -7.293935   
139       -7.396111      155.664862      10.151503   201.457351     -7.304351   
140       -7.396111      155.664862      10.151503   201.448593     -7.731080   
141       -7.396111      155.664862      10.151503   201.486077     -7.526079   

            b_J  
0     81.587075  
1    108.155070  
2     20.941453  
3     33.301944  
4     50.712957  
..          ...  
137   39.469849  
138   21.105844  
139   17.422224  
140   60.422810  
141   25.712282  

[142 rows x 24 columns]

### Find target stars

In [10]:
# threshold
thr = 5

# create threshold column
stars_data['thr'] = stars_data['b_J'] <= thr

# print number of targets
stars_data['thr'].value_counts()

False    133
True       9
Name: thr, dtype: int64

### Create target and reference dataframe

In [11]:
def ref_targ_split(df):
    
    filt = df['thr']

    target_df = df.loc[filt]
    reference_df = df.loc[(~filt)]
    
    return target_df, reference_df

## Evaluate deflection

In [12]:
# the positions are Gaiacentric thus we put the observer in the origin
x_obs = np.array([0, 0, 0])

In [13]:
# define epochs
epochs = stars_data['tAFx'].unique()

In [29]:
# differences
shell=[]

# loop over epochs
for epoch in epochs:
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    shell_epoch = np.zeros((len(target_df), len(reference_df)))

    # create body
    x_b = cartesian(stars_epoch['etaJup[deg]'].iloc[0], stars_epoch['zetaJup[deg]'].iloc[0]).T * jup_radius / np.deg2rad(r_jup_arc/3600)
    spin = cartesian(stars_epoch['etaSpin[deg]'].iloc[0], stars_epoch['zetaSpin[deg]'].iloc[0])
    
    body = Body(mass=jup_mass,
                pos=x_b,
                radius=jup_radius,
                J2=jupiter.J2,
                s=spin)
    
    # loop over target
    x_targ = cartesian(target_df['etaS[deg]'], target_df['zetaS[deg]']).T * pc
    for i, x in enumerate(x_targ):
        
        # direction
        l0 = -(x - x_obs) / np.linalg.norm(x - x_obs)
        
        # deflection
        dl = deflection(l0, x, body.pos, x_obs, eps, v_null, body.mass)
        defl_tar = np.linalg.norm(dl)
        print(np.rad2deg(defl_tar)*3600*1e6)
       
        # deflection quadrupole
        dl = deflection(l0, x, body.pos, x_obs, eps, v_null, body.mass, body.s, body.J2, body.radius)
        defl_q_tar = np.linalg.norm(dl)
        #print(np.rad2deg(defl_q_tar)*3600*1e6)
        
        # loop over reference
        x_ref = cartesian(reference_df['etaS[deg]'], reference_df['zetaS[deg]']).T * pc
        for j, y in enumerate(x_ref):
            
            # direction
            l0 = -(y - x_obs) / np.linalg.norm(y - x_obs)
        
            # deflection
            dl = deflection(l0, y, body.pos, x_obs, eps, v_null, body.mass)
            defl_ref = np.linalg.norm(dl)
            diff = defl_tar - defl_ref
            
            #print(epoch, np.rad2deg(defl_ref)*3600*1e6)
            print(epoch, np.rad2deg(diff)*3600*1e6)
            
            # deflection quadrupole
            dl = deflection(l0, y, body.pos, x_obs, eps, v_null, body.mass, body.s, body.J2, body.radius)
            defl_q_ref = np.linalg.norm(dl)
            diff_q = defl_q_tar - defl_q_ref
            
            #print(epoch, np.rad2deg(defl_q_ref)*3600*1e6)
            #print(epoch, np.rad2deg(diff_q)*3600*1e6)
            
            shell_epoch[i, j] = np.rad2deg(diff)*3600*1e6
    
    shell.append(shell_epoch)

3757.9058961533256
2017.1452385988 3553.968752702485
2017.1452385988 3604.065883391303
2017.1452385988 2963.3731326560455
2017.1452385988 3258.27552172858
2017.1452385988 3429.8112752632073
2017.1452385988 3607.2510876224496
2017.1452385988 3235.5302450755016
2017.1452385988 3473.9108681335524
2017.1452385988 2920.8983693413716
2017.1452385988 3368.016103106285
2017.1452385988 3336.3253224076907
2017.1452385988 2969.700690000865
2017.1452385988 2802.901410530999
2017.1452385988 3482.52631055265
2017.1452385988 3110.711667507699
3758.330996413581
2017.145238752673 3554.3944320406167
2017.145238752673 3604.4907327604174
2017.145238752673 2963.8170540145925
2017.145238752673 3258.6947098193255
2017.145238752673 3430.2345100060697
2017.145238752673 3607.676287789586
2017.145238752673 3235.9614441836384
2017.145238752673 3474.3362342965384
2017.145238752673 2921.317305066222
2017.145238752673 3368.44793836021
2017.145238752673 3336.7558167591565
2017.145238752673 2970.108242035235
2017.1452

In [32]:
shell

[array([[3553.9687527 , 3604.06588339, 2963.37313266, 3258.27552173,
         3429.81127526, 3607.25108762, 3235.53024508, 3473.91086813,
         2920.89836934, 3368.01610311, 3336.32532241, 2969.70069   ,
         2802.90141053, 3482.52631055, 3110.71166751]]),
 array([[3554.39443204, 3604.49073276, 2963.81705401, 3258.69470982,
         3430.23451001, 3607.67628779, 3235.96144418, 3474.3362343 ,
         2921.31730507, 3368.44793836, 3336.75581676, 2970.10824204,
         2803.32637768, 3482.952664  , 3111.14666661]]),
 array([[3554.76356124, 3604.85874381, 2964.2106166 , 3259.05642225,
         3430.60097528, 3608.04449971, 3236.33489026, 3474.70446988,
         2921.67358835, 3368.82196273, 3337.12728735, 2970.46009664,
         2803.69031704, 3483.32205364, 3111.53424467]]),
 array([[3555.32451928, 3605.41868696, 2964.79201289, 3259.61028274,
         3431.15921689, 3608.6046418 , 3236.90020338, 3475.2648779 ,
         2922.19047757, 3369.38590941, 3337.69050071, 2971.00624134,
 